In [28]:
import pandas as pd
import cv2
import os
from PIL import Image
import random
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
videos_path = "../../../ChimpRec-Dataset/CCR/data/videos"
annotations_path = "../../../ChimpRec-Dataset/CCR/annotations/face_data.csv"
dataset_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset"
train_path = f"{dataset_path}/train"
val_path = f"{dataset_path}/val"

In [30]:
annotations = pd.read_csv(annotations_path)

In [31]:
keeped_ind = ["JOYA", "FLANLE", "FANLE", "JEJE", "PELEY", "FANWA"]

In [32]:
for name in keeped_ind: 
    print(len(annotations.loc[annotations["label"] == name]))


59951
106096
149942
189896
74550
58635


In [33]:
train_set = {}
val_set = {}

for name in keeped_ind:
    individual_annotations = annotations[annotations["label"] == name]
    
    selected_samples = individual_annotations.sample(n=1000, random_state=42)

    train_samples, val_samples = train_test_split(selected_samples, test_size=0.1, random_state=42)
    
    train_set[name] = train_samples
    val_set[name] = val_samples

print(train_set)

{'JOYA':          year                     video   frame  track         x         y  \
262473   2012        20120202150041.mp4   12920    208  0.316978  0.228145   
121006   2012        20120201093157.mp4    7610    207  0.910153  0.400579   
196533   2012        20120201093157.mp4   39851   1495  0.743803  0.235253   
688993   2012        20120202150041.mp4  147639   3956  0.389984  0.719781   
1222196  2012  20120210142221_PAB20.mp4    8091    238  0.793144  0.258318   
...       ...                       ...     ...    ...       ...       ...   
108406   2012        20120201093157.mp4     988     23  0.661204  0.594388   
280500   2012        20120202150041.mp4   21494    435  0.671516  0.213948   
269064   2012        20120202150041.mp4   15216    230  0.561243  0.303439   
1222783  2012  20120210142221_PAB20.mp4    8680    242  0.513293  0.344040   
1244769  2012  20120210142221_PAB20.mp4   17475    431  0.493516  0.080117   

                w         h label  
262473   0.109348 

In [34]:
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

In [35]:
def get_frame(video_path, frame_number): 
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Impossible de lire la vidéo : {video_path}")
        return 
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_number >= total_frames:
        print(f"Frame {frame_number} hors limites pour la vidéo {video_path}. Total frames: {total_frames}")
        cap.release()
        return 

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()

    if not ret or frame is None or frame.size == 0:
        print(f"Erreur lors de la lecture de la frame {frame_number} pour {video_path}")
        return 
    return frame


def get_coordinate(x,y,w,h,img_W,img_H): 
    x = max(0, min(x, 1))
    y = max(0, min(y, 1))
    w = max(0, min(w, 1))
    h = max(0, min(h, 1))

    x1 = max(0, int(x * img_W))
    y1 = max(0, int(y * img_H))
    x2 = min(img_W, int((x + w) * img_W))
    y2 = min(img_H, int((y + h) * img_H))
    return x1,y1,x2,y2


def get_cropped_frame(frame, x1,y1,x2,y2,video_path,frame_number): 
    cropped_frame = frame[y1:y2, x1:x2]
    if cropped_frame.size == 0:
        print(f"Image vide pour {video_path}, frame {frame_number}")
        return 
    
    return Image.fromarray(cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB))


def save_images_to_structure(df, output_dir, name):
    num = 1
    for _, face in df.iterrows():
        year = face['year']
        video = face['video']
        frame_number = face['frame']
        x, y, w, h = face['x'], face['y'], face['w'], face['h']

        video_path = f"{videos_path}/{year}/{video}"
        frame = get_frame(video_path, frame_number)

        x1,y1,x2,y2 = get_coordinate(x,y,w,h,frame.shape[1],frame.shape[0])
        
        cropped_image = get_cropped_frame(frame, x1,y1,x2,y2,video_path,frame_number)

        image_path = f"{output_dir}/{name}_{num}.jpg"
        cropped_image.save(image_path)
        num += 1


for name, chimp_df in train_set.items():
    chimp_path = f"{train_path}/{name}"
    os.makedirs(chimp_path, exist_ok=True)
    save_images_to_structure(chimp_df, chimp_path, name)

for name, chimp_df in val_set.items():
    chimp_path = f"{val_path}/{name}"
    os.makedirs(chimp_path, exist_ok=True)
    save_images_to_structure(chimp_df, chimp_path, name)

In [36]:
"""csv_data = []

for index, face in final_df.iterrows():
    print(index)
    year = face['year']
    video = face['video']  
    frame_number = face['frame']
    track_id = face['track']
    x, y, w, h = face['x'], face['y'], face['w'], face['h']
    label = face['label']

    video_path = f"{videos_path}/{year}/{video}"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Impossible de lire la vidéo : {video_path}")
    else:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if frame_number >= total_frames:
            print(f"Frame {frame_number} est hors des limites pour la vidéo {video_path}. Total frames: {total_frames}")
            cap.release()
            continue
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        ret, frame = cap.read()

        if not ret:
            print(f"Impossible de lire la frame {frame_number} dans la vidéo {video_path}")
            cap.release()
            continue 

        cap.release()

        if frame is None or frame.size == 0:
            print(f"Frame vide lue pour {video_path} à la frame {frame_number}")
            continue 

        img_W, img_H = frame.shape[1], frame.shape[0]

        x = max(0, min(x, 1)) 
        y = max(0, min(y, 1)) 
        w = max(0, min(w, 1))
        h = max(0, min(h, 1))

        x1 = max(0, int(x * img_W))
        y1 = max(0, int(y * img_H))
        x2 = min(img_W, int((x + w) * img_W))
        y2 = min(img_H, int((y + h) * img_H))
        
        cropped_frame = frame[y1:y2, x1:x2]
            
        cropped_image = Image.fromarray(cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB))

        image_path = f"{dataset_path}/{index}.jpg"
        cropped_image.save(image_path)

        csv_data.append({
            'index': index,
            'label': label,
            'image_path': image_path
        })

csv_df = pd.DataFrame(csv_data)
csv_df.to_csv(f"{dataset_path}/video_face_mapping.csv", index=False)
print("CSV créé et images sauvegardées.")
"""

'csv_data = []\n\nfor index, face in final_df.iterrows():\n    print(index)\n    year = face[\'year\']\n    video = face[\'video\']  \n    frame_number = face[\'frame\']\n    track_id = face[\'track\']\n    x, y, w, h = face[\'x\'], face[\'y\'], face[\'w\'], face[\'h\']\n    label = face[\'label\']\n\n    video_path = f"{videos_path}/{year}/{video}"\n    cap = cv2.VideoCapture(video_path)\n    if not cap.isOpened():\n        print(f"Impossible de lire la vidéo : {video_path}")\n    else:\n        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\n        if frame_number >= total_frames:\n            print(f"Frame {frame_number} est hors des limites pour la vidéo {video_path}. Total frames: {total_frames}")\n            cap.release()\n            continue\n        \n        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)\n\n        ret, frame = cap.read()\n\n        if not ret:\n            print(f"Impossible de lire la frame {frame_number} dans la vidéo {video_path}")\n            c

In [38]:
"""for index, row in annotations.iterrows():
    video_path = row['ideo']  # Chemin ou identifiant de la vidéo
    frame_number = row['frame']  # Numéro de la frame
    track_id = row['track']"""

"for index, row in annotations.iterrows():\n    video_path = row['ideo']  # Chemin ou identifiant de la vidéo\n    frame_number = row['frame']  # Numéro de la frame\n    track_id = row['track']"